In [ ]:
!pip install -q datasets loralib sentencepiece
!pip uninstall transformers
!pip install -q git+https://github.com/zphang/transformers@c3dc391
!pip -q install git+https://github.com/huggingface/peft.git
!pip -q install bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 19.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 60

In [ ]:
from peft import PeftModel
from transformers import LLaMATokenizer, LLaMAForCausalLM, GenerationConfig
import textwrap



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('8013'), PosixPath('http'), PosixPath('//172.28.0.1')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//colab.research.google.com/tun/m/cc483011

In [ ]:

tokenizer = LLaMATokenizer.from_pretrained("decapoda-research/llama-7b-hf")

model = LLaMAForCausalLM.from_pretrained(
    "decapoda-research/llama-7b-hf",
    load_in_8bit=True,
    device_map="auto",
)
model = PeftModel.from_pretrained(model, "samwit/alpaca7B-lora")

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

In [ ]:

def alpaca_talk(text):
    inputs = tokenizer(
        text,
        return_tensors="pt",
    )
    input_ids = inputs["input_ids"].cuda()

    generation_config = GenerationConfig(
        temperature=0.6,
        top_p=0.7,
        repetition_penalty=1.2,
    )
    print("Generating...")
    generation_output = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=256,
    )
    for s in generation_output.sequences:
        print(tokenizer.decode(s))

In [ ]:
input_text ='''Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
I am going to give you two resumes and one job posting.Which one of the resumes is the best fit to the job posting?
Job:
 Kanary | Backend Engineer | Remote, Seattle | $120k base + generous equity
We’re scaling an automated security risk detection and data removal service for individuals. You might be a fit if you have experience doing CTF challenges, distributed systems work and/or web scraping. If you want to be on the ground level of building the most advanced automation platform for personal data security, privacy, and safety, keep reading.
As a backend-focused engineer, you’ll own projects involving infrastructure, devops, data flows, and APIs. But as an early startup teammate, you’ll work across all parts of our product, marketing, and strategy. Some examples:
* Using hardware and software to navigate the web anonymously * Using LLM agents to build internal tools for developers and support * Reverse engineer identification systems meant to harvest data and discriminate access
We’re a YC-backed team with previous startup, growth, and engineering experience at IBM, Shopify, Northwestern, Stanford, Tableau, and law enforcement. We’re backed by top VCs (2048, First Star) and privacy-leaders (Mozilla Foundation, DuckDuckGo). Together, we launched our product and 9x’d our membership - without paid advertising.
Tactically, our software runs on AWS and we use Django and React. Strong knowledge of Python, Go, and SQL is a plus. If you learn quickly and love the mission, but don’t have these core skills, please still reach out. We care most about people who are fantastic teammates over industry experts.
If this sounds like fun, say hi: rachel[at]kanary.com https://www.kanary.com/

Resume 1:
+90 546 671 2950 haticeobuz2001@hotmail.com
https://www.linkedin.com/in/hatice-obuz- 17b9a020a/
Manisa, Turkey
SKILLS
SQL
C Programming
Python
Circuit Design Assembly Language MATLAB
CERTIFICATIONS
TOEFL iBT
May, 2023 - May, 2025
Scored 100/120 on TOEFL iBT.
YDS-2/2021
Oct, 2021
Scored 91.25/100.
LANGUAGES
English Turkish
HATICE OBUZ
ELECTRONICS AND COMMUNICATIONS ENGINEER
EMPLOYMENT HISTORY
Long-Term Intern Garanti BBVA
July 2022 - December 2022
Worked as a long-term intern at Garanti BBVA for 6 months.
My internship was in the Department of Credit Risk and Advanced Analytics.
I completed many training programs such as Anger Management and Work Safety.
I worked under a mentor.
I participated in the process of adapting a vendor model to the bank while I was there.
I used SQL programming language to analyze customer tables.
I used Python programming language for the adaption of the vendor model.
Intern EmitelTR
I was an intern at the EmitelTR Test Laboratory.
I performed tests on electrical equipment
I learned how to read standards such as ISO-16750.
EDUCATION
Bachelor's of Science, Izmir Institute of Technology
Oct 2019 - Present
Planned Graduation Date: July, 2023 GPA: 2.93/4.00
Courses Taken:
Introduction to Digital Signal Processing Electronics I & II
Communication Systems I & II
Signals & Systems
Circuit Analysis I & II Behavioral Robotics Digital System Design Embedded Systems
June 2021- July 2021

Resume 2:
CV2: Afra
ildesafra@gmail.com • www.linkedin.com/in/afraildes • İzmir • 553 651 8663
Electronics and Communication Engineering B.S, I would like to have a job opportunity in a company that can respond to my desire for continuous learning, and contribute to my career and personal development, in line with my university and the various pieces of training I have received with my effort, and I would like to direct my career by benefiting from the experiences in your company.
Responsible, determined, enthusiastic, adaptable, flexible, collaborative, and innovative. Committed to continuously improving skills and processes.
Professional Proficiency in English.
Specialties and Technologies. Python, JavaScript, SQL, HTML, CSS, Machine Learning, Artificial Intelligence, Matlab, Simulink, C, Microsoft Office.
EDUCATION
Bachelor of Science (BS) in Electronics and Communication Engineering,
İzmir Institute of Technology, İzmir, 2018 – 2023 Science Class in High School,
Manisa Science High School, Manisa, 2014 – 2018
PROFESSIONAL EXPERIENCE
E.C.A VALF Sanayii A.Ş, Manisa Engineer Intern
Project realization and reporting with Arduino
 PCB design and BOM list creation
PROJECTS
Text Summarization with Natural Language Processing
Python, Jupyter Notebook, PyTorch, PyDataset, Tkinter. Brand Detection of Cars
Python, Jupyter Notebook, YOLO, PyTorch.
Information Extraction Using Natural Language Processing  Python, Google Colab, LLMs.
TRAINING & CERTIFICATIONS
Udemy, From Zero to Advanced Level Modern Javascript Lessons ES7+/ Present
BTK Akademi, Web Development with HTML5/ Certificate,2023 BTK Akademi, Learning SQL with Applications/ Certificate, 2023
BTK Akademi, Python and TensorFlow for Data Science/ Certificate, 2022 TOBB, ISO 27001 Information Security Management System / Certificate, 2022 BTK Akademi, Introduction to Information Technologies/ Certificate, 2021
Jul 2022 – Aug 2022
### Response:
'''

alpaca_talk(input_text)

Generating...
 Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
I am going to give you two resumes and one job posting.Which one of the resumes is the best fit to the job posting?
Job:
 Kanary | Backend Engineer | Remote, Seattle | $120k base + generous equity
We’re scaling an automated security risk detection and data removal service for individuals. You might be a fit if you have experience doing CTF challenges, distributed systems work and/or web scraping. If you want to be on the ground level of building the most advanced automation platform for personal data security, privacy, and safety, keep reading.
As a backend-focused engineer, you’ll own projects involving infrastructure, devops, data flows, and APIs. But as an early startup teammate, you’ll work across all parts of our product, marketing, and strategy. Some examples:
* Using hardware and software to navigate the web anonymously * Using LLM agents to 

In [ ]:
input_text ='''Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction: I am going to give you a job posting and two resumes. Rate the match between the job posting and "Resume 1" out of 100.
Job:
Kanary | Backend Engineer | Remote, Seattle | $120k base + generous equity
We’re scaling an automated security risk detection and data removal service for individuals. You might be a fit if you have experience doing CTF challenges, distributed systems work and/or web scraping. If you want to be on the ground level of building the most advanced automation platform for personal data security, privacy, and safety, keep reading.
As a backend-focused engineer, you’ll own projects involving infrastructure, devops, data flows, and APIs. But as an early startup teammate, you’ll work across all parts of our product, marketing, and strategy. Some examples:
* Using hardware and software to navigate the web anonymously * Using LLM agents to build internal tools for developers and support * Reverse engineer identification systems meant to harvest data and discriminate access
We’re a YC-backed team with previous startup, growth, and engineering experience at IBM, Shopify, Northwestern, Stanford, Tableau, and law enforcement. We’re backed by top VCs (2048, First Star) and privacy-leaders (Mozilla Foundation, DuckDuckGo). Together, we launched our product and 9x’d our membership - without paid advertising.
Tactically, our software runs on AWS and we use Django and React. Strong knowledge of Python, Go, and SQL is a plus. If you learn quickly and love the mission, but don’t have these core skills, please still reach out. We care most about people who are fantastic teammates over industry experts.
If this sounds like fun, say hi: rachel[at]kanary.com https://www.kanary.com/

Resume 1:
+90 546 671 2950 haticeobuz2001@hotmail.com
https://www.linkedin.com/in/hatice-obuz- 17b9a020a/
Manisa, Turkey
SKILLS
SQL
C Programming
Python
Circuit Design Assembly Language MATLAB
CERTIFICATIONS
TOEFL iBT
May, 2023 - May, 2025
Scored 100/120 on TOEFL iBT.
YDS-2/2021
Oct, 2021
Scored 91.25/100.
LANGUAGES
English Turkish
HATICE OBUZ
ELECTRONICS AND COMMUNICATIONS ENGINEER
EMPLOYMENT HISTORY
Long-Term Intern Garanti BBVA
July 2022 - December 2022
Worked as a long-term intern at Garanti BBVA for 6 months.
My internship was in the Department of Credit Risk and Advanced Analytics.
I completed many training programs such as Anger Management and Work Safety.
I worked under a mentor.
I participated in the process of adapting a vendor model to the bank while I was there.
I used SQL programming language to analyze customer tables.
I used Python programming language for the adaption of the vendor model.
Intern EmitelTR
I was an intern at the EmitelTR Test Laboratory.
I performed tests on electrical equipment
I learned how to read standards such as ISO-16750.
EDUCATION
Bachelor's of Science, Izmir Institute of Technology
Oct 2019 - Present
Planned Graduation Date: July, 2023 GPA: 2.93/4.00
Courses Taken:
Introduction to Digital Signal Processing Electronics I & II
Communication Systems I & II
Signals & Systems
Circuit Analysis I & II Behavioral Robotics Digital System Design Embedded Systems
June 2021- July 2021

Resume 2:
CV2: Afra
ildesafra@gmail.com • www.linkedin.com/in/afraildes • İzmir • 553 651 8663
Electronics and Communication Engineering B.S, I would like to have a job opportunity in a company that can respond to my desire for continuous learning, and contribute to my career and personal development, in line with my university and the various pieces of training I have received with my effort, and I would like to direct my career by benefiting from the experiences in your company.
Responsible, determined, enthusiastic, adaptable, flexible, collaborative, and innovative. Committed to continuously improving skills and processes.
Professional Proficiency in English.
Specialties and Technologies. Python, JavaScript, SQL, HTML, CSS, Machine Learning, Artificial Intelligence, Matlab, Simulink, C, Microsoft Office.
EDUCATION
Bachelor of Science (BS) in Electronics and Communication Engineering,
İzmir Institute of Technology, İzmir, 2018 – 2023 Science Class in High School,
Manisa Science High School, Manisa, 2014 – 2018
PROFESSIONAL EXPERIENCE
E.C.A VALF Sanayii A.Ş, Manisa Engineer Intern
Project realization and reporting with Arduino
 PCB design and BOM list creation
PROJECTS
Text Summarization with Natural Language Processing
Python, Jupyter Notebook, PyTorch, PyDataset, Tkinter. Brand Detection of Cars
Python, Jupyter Notebook, YOLO, PyTorch.
Information Extraction Using Natural Language Processing  Python, Google Colab, LLMs.
TRAINING & CERTIFICATIONS
Udemy, From Zero to Advanced Level Modern Javascript Lessons ES7+/ Present
BTK Akademi, Web Development with HTML5/ Certificate,2023 BTK Akademi, Learning SQL with Applications/ Certificate, 2023
BTK Akademi, Python and TensorFlow for Data Science/ Certificate, 2022 TOBB, ISO 27001 Information Security Management System / Certificate, 2022 BTK Akademi, Introduction to Information Technologies/ Certificate, 2021
Jul 2022 – Aug 2022

### Response:
'''

alpaca_talk(input_text)

Generating...
 Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction: I am going to give you a job posting and two resumes. Rate the match between the job posting and "Resume 1" out of 100.
Job:
Kanary | Backend Engineer | Remote, Seattle | $120k base + generous equity
We’re scaling an automated security risk detection and data removal service for individuals. You might be a fit if you have experience doing CTF challenges, distributed systems work and/or web scraping. If you want to be on the ground level of building the most advanced automation platform for personal data security, privacy, and safety, keep reading.
As a backend-focused engineer, you’ll own projects involving infrastructure, devops, data flows, and APIs. But as an early startup teammate, you’ll work across all parts of our product, marketing, and strategy. Some examples:
* Using hardware and software to navigate the web anonymously * Using LLM agents 

In [ ]:
input_text ='''Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction: I am going to give you a job posting and two resumes. Rate the match between the job posting and "Resume 2" out of 100.
Job:
Kanary | Backend Engineer | Remote, Seattle | $120k base + generous equity
We’re scaling an automated security risk detection and data removal service for individuals. You might be a fit if you have experience doing CTF challenges, distributed systems work and/or web scraping. If you want to be on the ground level of building the most advanced automation platform for personal data security, privacy, and safety, keep reading.
As a backend-focused engineer, you’ll own projects involving infrastructure, devops, data flows, and APIs. But as an early startup teammate, you’ll work across all parts of our product, marketing, and strategy. Some examples:
* Using hardware and software to navigate the web anonymously * Using LLM agents to build internal tools for developers and support * Reverse engineer identification systems meant to harvest data and discriminate access
We’re a YC-backed team with previous startup, growth, and engineering experience at IBM, Shopify, Northwestern, Stanford, Tableau, and law enforcement. We’re backed by top VCs (2048, First Star) and privacy-leaders (Mozilla Foundation, DuckDuckGo). Together, we launched our product and 9x’d our membership - without paid advertising.
Tactically, our software runs on AWS and we use Django and React. Strong knowledge of Python, Go, and SQL is a plus. If you learn quickly and love the mission, but don’t have these core skills, please still reach out. We care most about people who are fantastic teammates over industry experts.
If this sounds like fun, say hi: rachel[at]kanary.com https://www.kanary.com/

Resume 1:
+90 546 671 2950 haticeobuz2001@hotmail.com
https://www.linkedin.com/in/hatice-obuz- 17b9a020a/
Manisa, Turkey
SKILLS
SQL
C Programming
Python
Circuit Design Assembly Language MATLAB
CERTIFICATIONS
TOEFL iBT
May, 2023 - May, 2025
Scored 100/120 on TOEFL iBT.
YDS-2/2021
Oct, 2021
Scored 91.25/100.
LANGUAGES
English Turkish
HATICE OBUZ
ELECTRONICS AND COMMUNICATIONS ENGINEER
EMPLOYMENT HISTORY
Long-Term Intern Garanti BBVA
July 2022 - December 2022
Worked as a long-term intern at Garanti BBVA for 6 months.
My internship was in the Department of Credit Risk and Advanced Analytics.
I completed many training programs such as Anger Management and Work Safety.
I worked under a mentor.
I participated in the process of adapting a vendor model to the bank while I was there.
I used SQL programming language to analyze customer tables.
I used Python programming language for the adaption of the vendor model.
Intern EmitelTR
I was an intern at the EmitelTR Test Laboratory.
I performed tests on electrical equipment
I learned how to read standards such as ISO-16750.
EDUCATION
Bachelor's of Science, Izmir Institute of Technology
Oct 2019 - Present
Planned Graduation Date: July, 2023 GPA: 2.93/4.00
Courses Taken:
Introduction to Digital Signal Processing Electronics I & II
Communication Systems I & II
Signals & Systems
Circuit Analysis I & II Behavioral Robotics Digital System Design Embedded Systems
June 2021- July 2021

Resume 2:
CV2: Afra
ildesafra@gmail.com • www.linkedin.com/in/afraildes • İzmir • 553 651 8663
Electronics and Communication Engineering B.S, I would like to have a job opportunity in a company that can respond to my desire for continuous learning, and contribute to my career and personal development, in line with my university and the various pieces of training I have received with my effort, and I would like to direct my career by benefiting from the experiences in your company.
Responsible, determined, enthusiastic, adaptable, flexible, collaborative, and innovative. Committed to continuously improving skills and processes.
Professional Proficiency in English.
Specialties and Technologies. Python, JavaScript, SQL, HTML, CSS, Machine Learning, Artificial Intelligence, Matlab, Simulink, C, Microsoft Office.
EDUCATION
Bachelor of Science (BS) in Electronics and Communication Engineering,
İzmir Institute of Technology, İzmir, 2018 – 2023 Science Class in High School,
Manisa Science High School, Manisa, 2014 – 2018
PROFESSIONAL EXPERIENCE
E.C.A VALF Sanayii A.Ş, Manisa Engineer Intern
Project realization and reporting with Arduino
 PCB design and BOM list creation
PROJECTS
Text Summarization with Natural Language Processing
Python, Jupyter Notebook, PyTorch, PyDataset, Tkinter. Brand Detection of Cars
Python, Jupyter Notebook, YOLO, PyTorch.
Information Extraction Using Natural Language Processing  Python, Google Colab, LLMs.
TRAINING & CERTIFICATIONS
Udemy, From Zero to Advanced Level Modern Javascript Lessons ES7+/ Present
BTK Akademi, Web Development with HTML5/ Certificate,2023 BTK Akademi, Learning SQL with Applications/ Certificate, 2023
BTK Akademi, Python and TensorFlow for Data Science/ Certificate, 2022 TOBB, ISO 27001 Information Security Management System / Certificate, 2022 BTK Akademi, Introduction to Information Technologies/ Certificate, 2021
Jul 2022 – Aug 2022

### Response:
'''

alpaca_talk(input_text)

Generating...
 Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction: I am going to give you a job posting and two resumes. Rate the match between the job posting and "Resume 2" out of 100.
Job:
Kanary | Backend Engineer | Remote, Seattle | $120k base + generous equity
We’re scaling an automated security risk detection and data removal service for individuals. You might be a fit if you have experience doing CTF challenges, distributed systems work and/or web scraping. If you want to be on the ground level of building the most advanced automation platform for personal data security, privacy, and safety, keep reading.
As a backend-focused engineer, you’ll own projects involving infrastructure, devops, data flows, and APIs. But as an early startup teammate, you’ll work across all parts of our product, marketing, and strategy. Some examples:
* Using hardware and software to navigate the web anonymously * Using LLM agents 

In [ ]:
input_text ='''Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction: I am going to give you two resumes and one job posting.Which one of the resumes is the best fit to the job posting?

Job:
Exactag (https://exactag.com) | Software Engineer (Full Stack or Backend) | 100% Remote (Germany Only) | Full Time
Exactag is a marketing attribution platform assisting marketers in measuring and optimizing their advertising budget. We're seeking an engineer to join our web application development team. Your work will contribute to data analysis and campaign optimization for some of the most valuable German brands.

Our small, fully remote team stays connected via Slack to discuss solutions and tackle challenges together. The ideal candidate has experience with a JVM-based language (e.g., Java, Kotlin, Groovy) and React or is open to learning React if needed. We allocate 10% of work time for continuous learning and mutual growth. In this role, you'll make a hands-on impact and gain insights into all aspects of our web app, including data ETL pipelines with Druid, backend services, and frontend development. We maintain a friendly, open atmosphere in a flat organizational structure.

Tech stack: Kotlin, Spring Boot, Apache Druid, React, TypeScript

Details: https://exactag.jobs.personio.de/job/1071437?language=&displ...

Apply here: https://exactag.jobs.personio.de/job/1071437?language=&displ...

If you're interested, we'd love to hear from you!

Resume 1:
+90 546 671 2950 haticeobuz2001@hotmail.com
https://www.linkedin.com/in/hatice-obuz- 17b9a020a/
Manisa, Turkey
SKILLS
SQL
C Programming
Python
Circuit Design Assembly Language MATLAB
CERTIFICATIONS
TOEFL iBT
May, 2023 - May, 2025
Scored 100/120 on TOEFL iBT.
YDS-2/2021
Oct, 2021
Scored 91.25/100.
LANGUAGES
English Turkish
HATICE OBUZ
ELECTRONICS AND COMMUNICATIONS ENGINEER
EMPLOYMENT HISTORY
Long-Term Intern Garanti BBVA
July 2022 - December 2022
Worked as a long-term intern at Garanti BBVA for 6 months.
My internship was in the Department of Credit Risk and Advanced Analytics.
I completed many training programs such as Anger Management and Work Safety.
I worked under a mentor.
I participated in the process of adapting a vendor model to the bank while I was there.
I used SQL programming language to analyze customer tables.
I used Python programming language for the adaption of the vendor model.
Intern EmitelTR
I was an intern at the EmitelTR Test Laboratory.
I performed tests on electrical equipment
I learned how to read standards such as ISO-16750.
EDUCATION
Bachelor's of Science, Izmir Institute of Technology
Oct 2019 - Present
Planned Graduation Date: July, 2023 GPA: 2.93/4.00
Courses Taken:
Introduction to Digital Signal Processing Electronics I & II
Communication Systems I & II
Signals & Systems
Circuit Analysis I & II Behavioral Robotics Digital System Design Embedded Systems
June 2021- July 2021

Resume 2:
CV2: Afra
ildesafra@gmail.com • www.linkedin.com/in/afraildes • İzmir • 553 651 8663
Electronics and Communication Engineering B.S, I would like to have a job opportunity in a company that can respond to my desire for continuous learning, and contribute to my career and personal development, in line with my university and the various pieces of training I have received with my effort, and I would like to direct my career by benefiting from the experiences in your company.
Responsible, determined, enthusiastic, adaptable, flexible, collaborative, and innovative. Committed to continuously improving skills and processes.
Professional Proficiency in English.
Specialties and Technologies. Python, JavaScript, SQL, HTML, CSS, Machine Learning, Artificial Intelligence, Matlab, Simulink, C, Microsoft Office.
EDUCATION
Bachelor of Science (BS) in Electronics and Communication Engineering,
İzmir Institute of Technology, İzmir, 2018 – 2023 Science Class in High School,
Manisa Science High School, Manisa, 2014 – 2018
PROFESSIONAL EXPERIENCE
E.C.A VALF Sanayii A.Ş, Manisa Engineer Intern
Project realization and reporting with Arduino
 PCB design and BOM list creation
PROJECTS
Text Summarization with Natural Language Processing
Python, Jupyter Notebook, PyTorch, PyDataset, Tkinter. Brand Detection of Cars
Python, Jupyter Notebook, YOLO, PyTorch.
Information Extraction Using Natural Language Processing  Python, Google Colab, LLMs.
TRAINING & CERTIFICATIONS
Udemy, From Zero to Advanced Level Modern Javascript Lessons ES7+/ Present
BTK Akademi, Web Development with HTML5/ Certificate,2023 BTK Akademi, Learning SQL with Applications/ Certificate, 2023
BTK Akademi, Python and TensorFlow for Data Science/ Certificate, 2022 TOBB, ISO 27001 Information Security Management System / Certificate, 2022 BTK Akademi, Introduction to Information Technologies/ Certificate, 2021
Jul 2022 – Aug 2022

### Response:
'''

alpaca_talk(input_text)

Generating...
 Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction: I am going to give you two resumes and one job posting.Which one of the resumes is the best fit to the job posting?

Job:
Exactag (https://exactag.com) | Software Engineer (Full Stack or Backend) | 100% Remote (Germany Only) | Full Time
Exactag is a marketing attribution platform assisting marketers in measuring and optimizing their advertising budget. We're seeking an engineer to join our web application development team. Your work will contribute to data analysis and campaign optimization for some of the most valuable German brands.

Our small, fully remote team stays connected via Slack to discuss solutions and tackle challenges together. The ideal candidate has experience with a JVM-based language (e.g., Java, Kotlin, Groovy) and React or is open to learning React if needed. We allocate 10% of work time for continuous learning and mutual growth.

In [ ]:
input_text ='''Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction: I am going to give you two resumes and one job posting.Which one of the resumes is the best fit to the job posting?

Job:
We are looking for individuals who are eager to develop and apply their skills in ways that make a positive impact to communities. Here, you’ll be proud to be a part of a company known for putting safety and people first, combined with uncompromising integrity.
Job Description
Full Time Work Available. Quick Turnaround Hire!
As an installer, you will be responsible for all aspects of installing company products according to work order, manufacturer, quality and safety guidelines. Products include insulation, fireplaces, gutters, mirrors and shelving, garage doors, windows, shower enclosures, home wrap and any other products sold by the company.
You will be proud to be a part of a team where the average installer tenure is 10+ years!

Resume 1:
+90 546 671 2950 haticeobuz2001@hotmail.com
https://www.linkedin.com/in/hatice-obuz- 17b9a020a/
Manisa, Turkey
SKILLS
SQL
C Programming
Python
Circuit Design Assembly Language MATLAB
CERTIFICATIONS
TOEFL iBT
May, 2023 - May, 2025
Scored 100/120 on TOEFL iBT.
YDS-2/2021
Oct, 2021
Scored 91.25/100.
LANGUAGES
English Turkish
HATICE OBUZ
ELECTRONICS AND COMMUNICATIONS ENGINEER
EMPLOYMENT HISTORY
Long-Term Intern Garanti BBVA
July 2022 - December 2022
Worked as a long-term intern at Garanti BBVA for 6 months.
My internship was in the Department of Credit Risk and Advanced Analytics.
I completed many training programs such as Anger Management and Work Safety.
I worked under a mentor.
I participated in the process of adapting a vendor model to the bank while I was there.
I used SQL programming language to analyze customer tables.
I used Python programming language for the adaption of the vendor model.
Intern EmitelTR
I was an intern at the EmitelTR Test Laboratory.
I performed tests on electrical equipment
I learned how to read standards such as ISO-16750.
EDUCATION
Bachelor's of Science, Izmir Institute of Technology
Oct 2019 - Present
Planned Graduation Date: July, 2023 GPA: 2.93/4.00
Courses Taken:
Introduction to Digital Signal Processing Electronics I & II
Communication Systems I & II
Signals & Systems
Circuit Analysis I & II Behavioral Robotics Digital System Design Embedded Systems
June 2021- July 2021

Resume 2:
CV2: Afra
ildesafra@gmail.com • www.linkedin.com/in/afraildes • İzmir • 553 651 8663
Electronics and Communication Engineering B.S, I would like to have a job opportunity in a company that can respond to my desire for continuous learning, and contribute to my career and personal development, in line with my university and the various pieces of training I have received with my effort, and I would like to direct my career by benefiting from the experiences in your company.
Responsible, determined, enthusiastic, adaptable, flexible, collaborative, and innovative. Committed to continuously improving skills and processes.
Professional Proficiency in English.
Specialties and Technologies. Python, JavaScript, SQL, HTML, CSS, Machine Learning, Artificial Intelligence, Matlab, Simulink, C, Microsoft Office.
EDUCATION
Bachelor of Science (BS) in Electronics and Communication Engineering,
İzmir Institute of Technology, İzmir, 2018 – 2023 Science Class in High School,
Manisa Science High School, Manisa, 2014 – 2018
PROFESSIONAL EXPERIENCE
E.C.A VALF Sanayii A.Ş, Manisa Engineer Intern
Project realization and reporting with Arduino
 PCB design and BOM list creation
PROJECTS
Text Summarization with Natural Language Processing
Python, Jupyter Notebook, PyTorch, PyDataset, Tkinter. Brand Detection of Cars
Python, Jupyter Notebook, YOLO, PyTorch.
Information Extraction Using Natural Language Processing  Python, Google Colab, LLMs.
TRAINING & CERTIFICATIONS
Udemy, From Zero to Advanced Level Modern Javascript Lessons ES7+/ Present
BTK Akademi, Web Development with HTML5/ Certificate,2023 BTK Akademi, Learning SQL with Applications/ Certificate, 2023
BTK Akademi, Python and TensorFlow for Data Science/ Certificate, 2022 TOBB, ISO 27001 Information Security Management System / Certificate, 2022 BTK Akademi, Introduction to Information Technologies/ Certificate, 2021
Jul 2022 – Aug 2022

### Response:
'''

alpaca_talk(input_text)

Generating...
 Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction: I am going to give you two resumes and one job posting.Which one of the resumes is the best fit to the job posting?

Job:
We are looking for individuals who are eager to develop and apply their skills in ways that make a positive impact to communities. Here, you’ll be proud to be a part of a company known for putting safety and people first, combined with uncompromising integrity.
Job Description
Full Time Work Available. Quick Turnaround Hire!
As an installer, you will be responsible for all aspects of installing company products according to work order, manufacturer, quality and safety guidelines. Products include insulation, fireplaces, gutters, mirrors and shelving, garage doors, windows, shower enclosures, home wrap and any other products sold by the company.
You will be proud to be a part of a team where the average installer tenure is 10+ yea

In [ ]:
input_text ='''Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction: I am going to give you a job posting and two resumes. Rate the match between the job posting and "Resume 1" out of 100.

Job:
We are looking for individuals who are eager to develop and apply their skills in ways that make a positive impact to communities. Here, you’ll be proud to be a part of a company known for putting safety and people first, combined with uncompromising integrity.
Job Description
Full Time Work Available. Quick Turnaround Hire!
As an installer, you will be responsible for all aspects of installing company products according to work order, manufacturer, quality and safety guidelines. Products include insulation, fireplaces, gutters, mirrors and shelving, garage doors, windows, shower enclosures, home wrap and any other products sold by the company.
You will be proud to be a part of a team where the average installer tenure is 10+ years!

Resume 1:
+90 546 671 2950 haticeobuz2001@hotmail.com
https://www.linkedin.com/in/hatice-obuz- 17b9a020a/
Manisa, Turkey
SKILLS
SQL
C Programming
Python
Circuit Design Assembly Language MATLAB
CERTIFICATIONS
TOEFL iBT
May, 2023 - May, 2025
Scored 100/120 on TOEFL iBT.
YDS-2/2021
Oct, 2021
Scored 91.25/100.
LANGUAGES
English Turkish
HATICE OBUZ
ELECTRONICS AND COMMUNICATIONS ENGINEER
EMPLOYMENT HISTORY
Long-Term Intern Garanti BBVA
July 2022 - December 2022
Worked as a long-term intern at Garanti BBVA for 6 months.
My internship was in the Department of Credit Risk and Advanced Analytics.
I completed many training programs such as Anger Management and Work Safety.
I worked under a mentor.
I participated in the process of adapting a vendor model to the bank while I was there.
I used SQL programming language to analyze customer tables.
I used Python programming language for the adaption of the vendor model.
Intern EmitelTR
I was an intern at the EmitelTR Test Laboratory.
I performed tests on electrical equipment
I learned how to read standards such as ISO-16750.
EDUCATION
Bachelor's of Science, Izmir Institute of Technology
Oct 2019 - Present
Planned Graduation Date: July, 2023 GPA: 2.93/4.00
Courses Taken:
Introduction to Digital Signal Processing Electronics I & II
Communication Systems I & II
Signals & Systems
Circuit Analysis I & II Behavioral Robotics Digital System Design Embedded Systems
June 2021- July 2021

Resume 2:
CV2: Afra
ildesafra@gmail.com • www.linkedin.com/in/afraildes • İzmir • 553 651 8663
Electronics and Communication Engineering B.S, I would like to have a job opportunity in a company that can respond to my desire for continuous learning, and contribute to my career and personal development, in line with my university and the various pieces of training I have received with my effort, and I would like to direct my career by benefiting from the experiences in your company.
Responsible, determined, enthusiastic, adaptable, flexible, collaborative, and innovative. Committed to continuously improving skills and processes.
Professional Proficiency in English.
Specialties and Technologies. Python, JavaScript, SQL, HTML, CSS, Machine Learning, Artificial Intelligence, Matlab, Simulink, C, Microsoft Office.
EDUCATION
Bachelor of Science (BS) in Electronics and Communication Engineering,
İzmir Institute of Technology, İzmir, 2018 – 2023 Science Class in High School,
Manisa Science High School, Manisa, 2014 – 2018
PROFESSIONAL EXPERIENCE
E.C.A VALF Sanayii A.Ş, Manisa Engineer Intern
Project realization and reporting with Arduino
 PCB design and BOM list creation
PROJECTS
Text Summarization with Natural Language Processing
Python, Jupyter Notebook, PyTorch, PyDataset, Tkinter. Brand Detection of Cars
Python, Jupyter Notebook, YOLO, PyTorch.
Information Extraction Using Natural Language Processing  Python, Google Colab, LLMs.
TRAINING & CERTIFICATIONS
Udemy, From Zero to Advanced Level Modern Javascript Lessons ES7+/ Present
BTK Akademi, Web Development with HTML5/ Certificate,2023 BTK Akademi, Learning SQL with Applications/ Certificate, 2023
BTK Akademi, Python and TensorFlow for Data Science/ Certificate, 2022 TOBB, ISO 27001 Information Security Management System / Certificate, 2022 BTK Akademi, Introduction to Information Technologies/ Certificate, 2021
Jul 2022 – Aug 2022

### Response:
'''

alpaca_talk(input_text)

Generating...
 Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction: I am going to give you a job posting and two resumes. Rate the match between the job posting and "Resume 1" out of 100.

Job:
We are looking for individuals who are eager to develop and apply their skills in ways that make a positive impact to communities. Here, you’ll be proud to be a part of a company known for putting safety and people first, combined with uncompromising integrity.
Job Description
Full Time Work Available. Quick Turnaround Hire!
As an installer, you will be responsible for all aspects of installing company products according to work order, manufacturer, quality and safety guidelines. Products include insulation, fireplaces, gutters, mirrors and shelving, garage doors, windows, shower enclosures, home wrap and any other products sold by the company.
You will be proud to be a part of a team where the average installer tenure is 10+

In [ ]:
input_text ='''Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
I will give you three job postings and a resume. Sort the jobs from most relevant to least relevant for my resume.

Resume:
 ildesafra@gmail.com • www.linkedin.com/in/afraildes • İzmir • 553 651 8663
Electronics and Communication Engineering B.S, I would like to have a job opportunity in a company that can respond to my desire for continuous learning, and contribute to my career and personal development, in line with my university and the various pieces of training I have received with my effort, and I would like to direct my career by benefiting from the experiences in your company.
Responsible, determined, enthusiastic, adaptable, flexible, collaborative, and innovative. Committed to continuously improving skills and processes.
Professional Proficiency in English.
Specialties and Technologies. Python, JavaScript, SQL, HTML, CSS, Machine Learning, Artificial Intelligence, Matlab, Simulink, C, Microsoft Office.
EDUCATION
Bachelor of Science (BS) in Electronics and Communication Engineering,
İzmir Institute of Technology, İzmir, 2018 – 2023 Science Class in High School,
Manisa Science High School, Manisa, 2014 – 2018
PROFESSIONAL EXPERIENCE
E.C.A VALF Sanayii A.Ş, Manisa Engineer Intern
Project realization and reporting with Arduino
 PCB design and BOM list creation
PROJECTS
Text Summarization with Natural Language Processing
Python, Jupyter Notebook, PyTorch, PyDataset, Tkinter. Brand Detection of Cars
Python, Jupyter Notebook, YOLO, PyTorch.
Information Extraction Using Natural Language Processing  Python, Google Colab, LLMs.
TRAINING & CERTIFICATIONS
Udemy, From Zero to Advanced Level Modern Javascript Lessons ES7+/ Present
BTK Akademi, Web Development with HTML5/ Certificate,2023 BTK Akademi, Learning SQL with Applications/ Certificate, 2023
BTK Akademi, Python and TensorFlow for Data Science/ Certificate, 2022 TOBB, ISO 27001 Information Security Management System / Certificate, 2022 BTK Akademi, Introduction to Information Technologies/ Certificate, 2021
Jul 2022 – Aug 2022 .


Job 1:

Merry is an early-stage social impact start-up that is building a product aimed at helping millions of Indian smallholder farmers improve the productivity of their farms. We're about to launch our product to farmers (take a look here: www.merry.in) and are in the process of building out our initial team, including our first tech-hire:
High level details of full-stack developer role:

  - Tech stack: Flutter, Flask, Python, Postgres database
  - ONSITE in Bangalore (2 days a week - rest is flexi)
  - Min 5 years experience
  - Details are here: https://docs.google.com/document/d/1usLzsFI3DK1mLXc-OLyCiagMU7Ngq8Ff7pRdlHHLrVw/edit#
If you want to have a forming role in building a product that can change the lives of millions, please reach out at jobs@merry.in

Job 2:

Outside, Inc | Sr. Cloud & Geospatial Engineer | REMOTE | $140k to $175k base + Equity
At Outside, we believe life is best spent outside experiencing healthy, connected, and fulfilling lives. We inspire and enable people to discover why.

Outside is the premier destination for active lifestyle enthusiasts. In addition to traditional, well-known media properties, Outside offers members a comprehensive suite of tools, experiences and content to help pursue their passions.

We're looking for an engineer to work on the geospatial backend behind our GaiaGPS and Trailforks products. We're looking for someone with experience in geospatial along with Python and AWS.

You can apply here:

https://outside.breezy.hr/p/d0ae624464bd-senior-engineer-clo...

Or feel free to ask me questions: msisk [at] outsideinc [dot] com

Job 3:

Ramp | Full-Time | Front-End Engineer (All Levels) | Remote Anywhere (NYC offices) | $250k+ TC Ramp is looking for talented front-end folks who live and breathe UX Our front-end stack is React, TypeScript, and a shiny component library.
Email your resume to nico@ramp.com.

See the job posting at https://boards.greenhouse.io/ramp/jobs/6658041002

We're also hiring for other roles, see all openings at https://grnh.se/660e10d82us. Particularly interested in Android (Kotlin), iOS (SwiftUI), and Backend (Python) engineers as well.

Ramp is building the world's first finance automation platform designed to save businesses time and money. Ramp offers 5-in-1 software that consolidates corporate cards, expense management, bill payments, accounting, and reporting into one simple and free solution. We're a young hypergrowth startup building top tier product in the B2B fintech space (3y old, $8.1B valuation, Series C, ~500 employees, ~100 engineers)

Cool breakdown of our business: https://www.notboring.co/p/ramping-up

More recent article on the economy: https://ramp.com/blog/ramp-july-2022-update

### Response:
'''

alpaca_talk(input_text)

Generating...
 Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction: 
I will give you three job postings and a resume. Sort the jobs from most relevant to least relevant for my resume.

Resume:
 ildesafra@gmail.com • www.linkedin.com/in/afraildes • İzmir • 553 651 8663 
Electronics and Communication Engineering B.S, I would like to have a job opportunity in a company that can respond to my desire for continuous learning, and contribute to my career and personal development, in line with my university and the various pieces of training I have received with my effort, and I would like to direct my career by benefiting from the experiences in your company. 
Responsible, determined, enthusiastic, adaptable, flexible, collaborative, and innovative. Committed to continuously improving skills and processes. 
Professional Proficiency in English. 
Specialties and Technologies. Python, JavaScript, SQL, HTML, CSS, Machine Lear

In [ ]:
input_text ='''Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
I will give you three job postings and a resume. Sort the jobs from most relevant to least relevant for my resume.

Resume:
 +90 546 671 2950 haticeobuz2001@hotmail.com
https://www.linkedin.com/in/hatice-obuz- 17b9a020a/
Manisa, Turkey
SKILLS
SQL
C Programming
Python
Circuit Design Assembly Language MATLAB
CERTIFICATIONS
TOEFL iBT
May, 2023 - May, 2025
Scored 100/120 on TOEFL iBT.
YDS-2/2021
Oct, 2021
Scored 91.25/100.
LANGUAGES
English Turkish
HATICE OBUZ
ELECTRONICS AND COMMUNICATIONS ENGINEER
EMPLOYMENT HISTORY
Long-Term Intern Garanti BBVA
July 2022 - December 2022
Worked as a long-term intern at Garanti BBVA for 6 months.
My internship was in the Department of Credit Risk and Advanced Analytics.
I completed many training programs such as Anger Management and Work Safety.
I worked under a mentor.
I participated in the process of adapting a vendor model to the bank while I was there.
I used SQL programming language to analyze customer tables.
I used Python programming language for the adaption of the vendor model.
Intern EmitelTR
I was an intern at the EmitelTR Test Laboratory.
I performed tests on electrical equipment
I learned how to read standards such as ISO-16750.
EDUCATION
Bachelor's of Science, Izmir Institute of Technology
Oct 2019 - Present
Planned Graduation Date: July, 2023 GPA: 2.93/4.00
Courses Taken:
Introduction to Digital Signal Processing Electronics I & II
Communication Systems I & II
Signals & Systems
Circuit Analysis I & II Behavioral Robotics Digital System Design Embedded Systems
June 2021- July 2021


Job 1:

Merry is an early-stage social impact start-up that is building a product aimed at helping millions of Indian smallholder farmers improve the productivity of their farms. We're about to launch our product to farmers (take a look here: www.merry.in) and are in the process of building out our initial team, including our first tech-hire:
High level details of full-stack developer role:

  - Tech stack: Flutter, Flask, Python, Postgres database
  - ONSITE in Bangalore (2 days a week - rest is flexi)
  - Min 5 years experience
  - Details are here: https://docs.google.com/document/d/1usLzsFI3DK1mLXc-OLyCiagMU7Ngq8Ff7pRdlHHLrVw/edit#
If you want to have a forming role in building a product that can change the lives of millions, please reach out at jobs@merry.in

Job 2:

Outside, Inc | Sr. Cloud & Geospatial Engineer | REMOTE | $140k to $175k base + Equity
At Outside, we believe life is best spent outside experiencing healthy, connected, and fulfilling lives. We inspire and enable people to discover why.

Outside is the premier destination for active lifestyle enthusiasts. In addition to traditional, well-known media properties, Outside offers members a comprehensive suite of tools, experiences and content to help pursue their passions.

We're looking for an engineer to work on the geospatial backend behind our GaiaGPS and Trailforks products. We're looking for someone with experience in geospatial along with Python and AWS.

You can apply here:

https://outside.breezy.hr/p/d0ae624464bd-senior-engineer-clo...

Or feel free to ask me questions: msisk [at] outsideinc [dot] com

Job 3:

Ramp | Full-Time | Front-End Engineer (All Levels) | Remote Anywhere (NYC offices) | $250k+ TC Ramp is looking for talented front-end folks who live and breathe UX Our front-end stack is React, TypeScript, and a shiny component library.
Email your resume to nico@ramp.com.

See the job posting at https://boards.greenhouse.io/ramp/jobs/6658041002

We're also hiring for other roles, see all openings at https://grnh.se/660e10d82us. Particularly interested in Android (Kotlin), iOS (SwiftUI), and Backend (Python) engineers as well.

Ramp is building the world's first finance automation platform designed to save businesses time and money. Ramp offers 5-in-1 software that consolidates corporate cards, expense management, bill payments, accounting, and reporting into one simple and free solution. We're a young hypergrowth startup building top tier product in the B2B fintech space (3y old, $8.1B valuation, Series C, ~500 employees, ~100 engineers)

Cool breakdown of our business: https://www.notboring.co/p/ramping-up

More recent article on the economy: https://ramp.com/blog/ramp-july-2022-update

### Response:
'''

alpaca_talk(input_text)

Generating...
 Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction: 
I will give you three job postings and a resume. Sort the jobs from most relevant to least relevant for my resume.

Resume:
 +90 546 671 2950 haticeobuz2001@hotmail.com 
https://www.linkedin.com/in/hatice-obuz- 17b9a020a/
Manisa, Turkey 
SKILLS 
SQL
C Programming 
Python
Circuit Design Assembly Language MATLAB 
CERTIFICATIONS 
TOEFL iBT 
May, 2023 - May, 2025
Scored 100/120 on TOEFL iBT. 
YDS-2/2021 
Oct, 2021
Scored 91.25/100. 
LANGUAGES 
English Turkish 
HATICE OBUZ 
ELECTRONICS AND COMMUNICATIONS ENGINEER 
EMPLOYMENT HISTORY 
Long-Term Intern Garanti BBVA 
July 2022 - December 2022 
Worked as a long-term intern at Garanti BBVA for 6 months.
My internship was in the Department of Credit Risk and Advanced Analytics.
I completed many training programs such as Anger Management and Work Safety.
I worked under a mentor.
I participated in the process o

In [ ]:
input_text ='''Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
I will give you three job postings and my resume. Find the most irrelevant job for my resume.

Resume:
 +90 546 671 2950 haticeobuz2001@hotmail.com
https://www.linkedin.com/in/hatice-obuz- 17b9a020a/
Manisa, Turkey
SKILLS
SQL
C Programming
Python
Circuit Design Assembly Language MATLAB
CERTIFICATIONS
TOEFL iBT
May, 2023 - May, 2025
Scored 100/120 on TOEFL iBT.
YDS-2/2021
Oct, 2021
Scored 91.25/100.
LANGUAGES
English Turkish
HATICE OBUZ
ELECTRONICS AND COMMUNICATIONS ENGINEER
EMPLOYMENT HISTORY
Long-Term Intern Garanti BBVA
July 2022 - December 2022
Worked as a long-term intern at Garanti BBVA for 6 months.
My internship was in the Department of Credit Risk and Advanced Analytics.
I completed many training programs such as Anger Management and Work Safety.
I worked under a mentor.
I participated in the process of adapting a vendor model to the bank while I was there.
I used SQL programming language to analyze customer tables.
I used Python programming language for the adaption of the vendor model.
Intern EmitelTR
I was an intern at the EmitelTR Test Laboratory.
I performed tests on electrical equipment
I learned how to read standards such as ISO-16750.
EDUCATION
Bachelor's of Science, Izmir Institute of Technology
Oct 2019 - Present
Planned Graduation Date: July, 2023 GPA: 2.93/4.00
Courses Taken:
Introduction to Digital Signal Processing Electronics I & II
Communication Systems I & II
Signals & Systems
Circuit Analysis I & II Behavioral Robotics Digital System Design Embedded Systems
June 2021- July 2021


Job 1:

Merry is an early-stage social impact start-up that is building a product aimed at helping millions of Indian smallholder farmers improve the productivity of their farms. We're about to launch our product to farmers (take a look here: www.merry.in) and are in the process of building out our initial team, including our first tech-hire:
High level details of full-stack developer role:

  - Tech stack: Flutter, Flask, Python, Postgres database
  - ONSITE in Bangalore (2 days a week - rest is flexi)
  - Min 5 years experience
  - Details are here: https://docs.google.com/document/d/1usLzsFI3DK1mLXc-OLyCiagMU7Ngq8Ff7pRdlHHLrVw/edit#
If you want to have a forming role in building a product that can change the lives of millions, please reach out at jobs@merry.in

Job 2:

We are looking for individuals who are eager to develop and apply their skills in ways that make a positive impact to communities. Here, you’ll be proud to be a part of a company known for putting safety and people first, combined with uncompromising integrity.
Job Description
Full Time Work Available. Quick Turnaround Hire!
As an installer, you will be responsible for all aspects of installing company products according to work order, manufacturer, quality and safety guidelines. Products include insulation, fireplaces, gutters, mirrors and shelving, garage doors, windows, shower enclosures, home wrap and any other products sold by the company.
You will be proud to be a part of a team where the average installer tenure is 10+ years!

Job 3:

Ramp | Full-Time | Front-End Engineer (All Levels) | Remote Anywhere (NYC offices) | $250k+ TC Ramp is looking for talented front-end folks who live and breathe UX Our front-end stack is React, TypeScript, and a shiny component library.
Email your resume to nico@ramp.com.

See the job posting at https://boards.greenhouse.io/ramp/jobs/6658041002

We're also hiring for other roles, see all openings at https://grnh.se/660e10d82us. Particularly interested in Android (Kotlin), iOS (SwiftUI), and Backend (Python) engineers as well.

Ramp is building the world's first finance automation platform designed to save businesses time and money. Ramp offers 5-in-1 software that consolidates corporate cards, expense management, bill payments, accounting, and reporting into one simple and free solution. We're a young hypergrowth startup building top tier product in the B2B fintech space (3y old, $8.1B valuation, Series C, ~500 employees, ~100 engineers)

Cool breakdown of our business: https://www.notboring.co/p/ramping-up

More recent article on the economy: https://ramp.com/blog/ramp-july-2022-update

### Response:
'''

alpaca_talk(input_text)

Generating...
 Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction: 
I will give you three job postings and my resume. Find the most irrelevant job for my resume.

Resume:
 +90 546 671 2950 haticeobuz2001@hotmail.com 
https://www.linkedin.com/in/hatice-obuz- 17b9a020a/
Manisa, Turkey 
SKILLS 
SQL
C Programming 
Python
Circuit Design Assembly Language MATLAB 
CERTIFICATIONS 
TOEFL iBT 
May, 2023 - May, 2025
Scored 100/120 on TOEFL iBT. 
YDS-2/2021 
Oct, 2021
Scored 91.25/100. 
LANGUAGES 
English Turkish 
HATICE OBUZ 
ELECTRONICS AND COMMUNICATIONS ENGINEER 
EMPLOYMENT HISTORY 
Long-Term Intern Garanti BBVA 
July 2022 - December 2022 
Worked as a long-term intern at Garanti BBVA for 6 months.
My internship was in the Department of Credit Risk and Advanced Analytics.
I completed many training programs such as Anger Management and Work Safety.
I worked under a mentor.
I participated in the process of adapting a vendor m